In [1]:
from PIL import Image
import torch
import torchvision
import torchvision.transforms as transforms

In [31]:
model = torchvision.models.resnet50(pretrained=False)

normalize = transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225))
inv_normalize = transforms.Normalize(mean=(-0.485/0.229, -0.456/0.224, -0.406/0.225),
                                   std=(1/0.229, 1/0.224, 1/0.225))
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [42]:
import numpy as np
import matplotlib.pyplot as plt

def saliency(img, model):
    
    for param in model.parameters():
        param.requires_grad = False
    
    model.eval()
    source = transform(img)
    source.unsqueeze_(0)
    
    source.reguires_grad = True
    preds = model(source)
    score, indices = torch.max(preds, 1)
    score.requires_grad = True
    score.backward()
    print(source.grad.shape)
    
    ta = torch.abs(source.grad[0])
    slc, _ = torch.max(ta, dim=0)
    
    slc = (slc - slc.min())/(slc.max() - slc.min())
    
    with torch.no_grad():
        input_img = inv_normalize(source[0])
    
    plt.figure(figsize=(10, 10))
    plt.subplot(1, 2, 1)
    plt.imshow(np.transpose(input_img.detach().numpy()), (1, 2, 0))
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1, 2, 2)
    plt.imshow(slc.numpy(), cmap=plt.cm.hot)
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [33]:
import os
from glob import glob
path = glob(os.path.join('/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/ART/','*'))



In [34]:
data = Image.open(path[0])
data = np.array(data)[:,:,:3]
data = Image.fromarray(data)

In [43]:
saliency(data, model)

AttributeError: 'NoneType' object has no attribute 'shape'